In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
df = pd.read_csv('final_bfsi_lablel.csv')

In [2]:
df

,Financial_context,annotation_id,annotator,content,created_at,id,language,lead_time,score,sentiment,thumbsUpCount,updated_at
0,Financial,14,1,1] Customer service is really nice.,2024-03-31T06:58:47.318794Z,1332,en,9.348,3,Positive,0,2024-03-31T06:58:47.318813Z
1,Non Financial,15,1,2] It is not very easy to set up mobile bankin...,2024-03-31T06:59:03.859506Z,1333,en,12.568,3,Neutral,0,2024-03-31T06:59:03.859527Z
2,Non Financial,16,1,I understand it's for safety but still a hassle.,2024-03-31T06:59:46.583294Z,1334,en,21.842,3,Neutral,0,2024-03-31T06:59:46.583307Z
3,Non Financial,17,1,3] App itself is easy to use.,2024-03-31T06:59:58.593876Z,1335,en,7.949,3,Positive,0,2024-03-31T06:59:58.593889Z
4,Non Financial,18,1,"4] Unfortunately, you can't view your card num...",2024-03-31T07:00:26.848093Z,1336,en,16.955,3,Negative,0,2024-03-31T07:00:26.848109Z
...,...,...,...,...,...,...,...,...,...,...,...,...
383,Non Financial,382,1,I don't know what's is the problem?,2024-03-31T11:39:35.667314Z,2532,en,9.276,4,Negative,0,2024-03-31T11:39:35.667327Z
384,Financial,383,1,It's very disappointing.,2024-03-31T11:39:43.659762Z,2533,en,6.336,4,Negative,0,2024-03-31T11:39:43.659780Z
385,Non Financial,370,1,2nd largest bank in India but mobile app is no...,2024-03-31T11:29:04.279001Z,2534,en,4.080,4,Negative,0,2024-03-31T11:29:04.279018Z
386,Financial,384,1,What a pity.,2024-03-31T11:40:03.224609Z,2535,en,25.356,4,Negative,0,2024-03-31T11:40:03.224623Z


In [3]:
final_df = df[['Financial_context','sentiment','content']]

In [4]:
final_df

,Financial_context,sentiment,content
0,Financial,Positive,1] Customer service is really nice.
1,Non Financial,Neutral,2] It is not very easy to set up mobile bankin...
2,Non Financial,Neutral,I understand it's for safety but still a hassle.
3,Non Financial,Positive,3] App itself is easy to use.
4,Non Financial,Negative,"4] Unfortunately, you can't view your card num..."
...,...,...,...
383,Non Financial,Negative,I don't know what's is the problem?
384,Financial,Negative,It's very disappointing.
385,Non Financial,Negative,2nd largest bank in India but mobile app is no...
386,Financial,Negative,What a pity.


In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Set aside a portion of the data for evaluation
eval_fraction = 0.1  # For example, 10% of the data for evaluation
train_test_data, X_eval = train_test_split(final_df, test_size=eval_fraction, random_state=42)

X_train = []
X_test = []

for sentiment in ["Positive", "Neutral", "Negative"]:
    subset = train_test_data[train_test_data['sentiment'] == sentiment]

    # Check the size of the subset
    subset_size = len(subset)

    # Set train and test size dynamically based on the subset size
    train_size = min(300, int(subset_size * 0.7))  # For example, 70% for training
    test_size = min(100, subset_size - train_size)  # Remaining for testing, but not more than 100

    # Perform the split
    if subset_size > 0:  # Proceed only if the subset is not empty
        train, test = train_test_split(subset,
                                       train_size=train_size,
                                       test_size=test_size,
                                       random_state=42)
        X_train.append(train)
        X_test.append(test)

# Concatenate the lists to form the full training and testing sets
if X_train and X_test:
    X_train = pd.concat(X_train).sample(frac=1, random_state=10).reset_index(drop=True)
    X_test = pd.concat(X_test).reset_index(drop=True)
    # For evaluation, you might want to ensure it's not overlapping with train/test
else:
    print("No data available for training and testing sets.")

remaining_idx = [idx for idx in df.index if idx not in X_train.index and idx not in X_test.index]
X_eval = df.loc[remaining_idx]

# If you want to sample from X_eval for balanced sentiment representation
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(min(len(x), 50), random_state=10, replace=True)))
X_eval = X_eval.reset_index(drop=True)

In [37]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the statement enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "Positive", "Neutral", or "Negative".
            Also, predict whether the statement is Financial or Non-Financial.

           Statement = [{data_point['content']}]

           Sentiment = {data_point['sentiment']}

           Financial Status = {data_point['Financial_context']}

            """.strip() + EOS_TOKEN

In [38]:
def generate_test_prompt(data_point):
    return f"""
           Analyze the sentiment of the statement enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "Positive" or "Neutral" or "Negative"

         Statement = [{data_point['content']}]

          Sentiment = {data_point['sentiment']}

           Financial Status = {data_point['Financial_context']}

    """.strip()

In [95]:
X_train['prompt'] = X_train.apply(generate_prompt, axis=1)
X_test['prompt'] = X_test.apply(generate_test_prompt, axis=1)
X_eval['prompt'] = X_eval.apply(generate_prompt, axis=1)

In [96]:
y_true = X_test[['sentiment', 'Financial_context']]
y_true_sentiment = X_test['sentiment']
y_true_financial = X_test['Financial_context']

In [97]:
pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["prompt"])

,prompt
0,Analyze the sentiment of the statement enclose...
1,Analyze the sentiment of the statement enclose...
2,Analyze the sentiment of the statement enclose...
3,Analyze the sentiment of the statement enclose...
4,Analyze the sentiment of the statement enclose...
...,...
238,Analyze the sentiment of the statement enclose...
239,Analyze the sentiment of the statement enclose...
240,Analyze the sentiment of the statement enclose...
241,Analyze the sentiment of the statement enclose...


In [16]:
!pip install -q -U datasets

In [42]:
from datasets import Dataset

train_data = Dataset.from_pandas(X_train[['prompt']])
eval_data = Dataset.from_pandas(X_eval[['prompt']])
test_data = Dataset.from_pandas(X_test[['prompt']])

In [18]:
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
import numpy as np
def evaluate(y_true, y_pred):
    labels = ['Positive', 'Neutral', 'Negative']
    mapping = {'Positive': 2, 'Neutral': 1,'none':1, 'Negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [92]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate(y_true_sentiment, y_pred_sentiment, y_true_financial, y_pred_financial):
    # Evaluation for sentiment
    sentiment_labels = ['Positive', 'Neutral', 'Negative']
    sentiment_mapping = {'Positive': 2, 'Neutral': 1, 'Negative': 0}

    y_true_sentiment_mapped = np.vectorize(sentiment_mapping.get)(y_true_sentiment)
    y_pred_sentiment_mapped = np.vectorize(sentiment_mapping.get)(y_pred_sentiment)

    print("Sentiment Analysis")
    print("------------------")
    print('Accuracy:', accuracy_score(y_true_sentiment_mapped, y_pred_sentiment_mapped))
    print('\nClassification Report:')
    print(classification_report(y_true_sentiment_mapped, y_pred_sentiment_mapped, target_names=sentiment_labels))
    print('Confusion Matrix:')
    print(confusion_matrix(y_true_sentiment_mapped, y_pred_sentiment_mapped, labels=[2, 1, 0]))

    # Evaluation for financial status
    financial_labels = ['Financial', 'Non Financial']
    financial_mapping = {'Financial': 1, 'Non Financial': 0}

    y_true_financial_mapped = np.vectorize(financial_mapping.get)(y_true_financial)
    y_pred_financial_mapped = np.vectorize(financial_mapping.get)(y_pred_financial)

    print("\nFinancial Status Analysis")
    print("-------------------------")
    print('Accuracy:', accuracy_score(y_true_financial_mapped, y_pred_financial_mapped))
    print('\nClassification Report:')
    print(classification_report(y_true_financial_mapped, y_pred_financial_mapped, target_names=financial_labels))
    print('Confusion Matrix:')
    print(confusion_matrix(y_true_financial_mapped, y_pred_financial_mapped, labels=[1, 0]))

In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)


model = AutoModelForCausalLM.from_pretrained(
   'google/gemma-2b',
    device_map="auto",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained('google/gemma-2b', max_seq_length=max_seq_length)
EOS_TOKEN = tokenizer.eos_token

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [19]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [20]:
modules = find_all_linear_names(model)
print(modules)

['v_proj', 'q_proj', 'down_proj', 'o_proj', 'k_proj', 'up_proj', 'gate_proj']


In [87]:
from tqdm import tqdm

def predict(X_test, model, tokenizer):
    y_pred_sentiment = []
    y_pred_financial = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["prompt"]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1, temperature=0.0)
        result = tokenizer.decode(outputs[0])

        # Sentiment analysis
        sentiment = "none"  # Default sentiment
        parts = result.split("Sentiment =")
        if len(parts) > 1:
            sentiment_part = parts[1].split("\n")[0].strip()
            if "positive" in sentiment_part.lower():
                sentiment = "Positive"
            elif "negative" in sentiment_part.lower():
                sentiment = "Negative"
            elif "neutral" in sentiment_part.lower():
                sentiment = "Neutral"
        y_pred_sentiment.append(sentiment)

        # Extract financial status
        financial = "none"
        financial_parts = result.split(" Financial Status = ")
        if len(financial_parts) > 1:
            financial_part = financial_parts[1].split("\n")[0].strip()
            if "non financial" in financial_part.lower():
                financial = "Non Financial"
            elif "financial" in financial_part.lower():
                financial = "Financial"
        y_pred_financial.append(financial)

    return y_pred_sentiment, y_pred_financial

In [90]:
y_pred_sentiment,y_pred_financial =predict(X_test, model, tokenizer)

100%|██████████| 106/106 [00:11<00:00,  9.32it/s]


In [98]:
evaluate(y_true_sentiment, y_pred_sentiment, y_true_financial, y_pred_financial)

Sentiment Analysis
------------------
Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

    Positive       1.00      1.00      1.00        45
     Neutral       1.00      1.00      1.00         9
    Negative       1.00      1.00      1.00        52

    accuracy                           1.00       106
   macro avg       1.00      1.00      1.00       106
weighted avg       1.00      1.00      1.00       106

Confusion Matrix:
[[52  0  0]
 [ 0  9  0]
 [ 0  0 45]]

Financial Status Analysis
-------------------------
Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

    Financial       1.00      1.00      1.00        43
Non Financial       1.00      1.00      1.00        63

     accuracy                           1.00       106
    macro avg       1.00      1.00      1.00       106
 weighted avg       1.00      1.00      1.00       106

Confusion Matrix:
[[63  0]
 [ 0 43]]
